# Batch processing of beam current and power, as well as critical current and critical temperature measurements

Here a "beam on event" refers to three Ic measurements, taken before, during and after exposure to a proton beam. The beam current is measured simulaneously, and a representative values of beam energy is obtained from lab notes (accurate within 5 keV).

**IMPORTANT NOTE:** This spreadsheet only needs to be run to incorporate new irradiation data into the data set.

In [57]:
%pylab notebook
import os, sys, importlib, warnings
warnings.filterwarnings("ignore")
sys.path.append('lib/')
import pandas as pd, numpy as np, scipy as scp , seaborn as sns
import hts_fitting as hts, default_figure_style as dfs, hts_dosecalculations as dose, hts_fitfunctions as ff, hts_misc as misc
import hts_beamspot as bsp

for lib in [hts, ff, dfs, misc]:
    importlib.reload(lib)

from ipywidgets import IntProgress, IntText
from IPython.display import display

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


## Step 1: extract the mean and stdev of beam current for each "beam-on event"

Here the code will extract (**from data-master.xlsx**) the user input time intervals where beam current and offset should be read (from the **data** sheet) for each event, producing averages and standard deviations. These values are then written back into **output-icnt** sheet. Within the excel sheet, these values are copied back into the **data** sheet, which can be read altogether for data analysis.

In [2]:
cols = [0, 3, 4, 5, 6, 7]

names = [
    'tapeid',     # tape sample name
    'sname',      # sheet name in spreadsheet with beam current data
    't0Beam',     # start time of interval for beam current evaluation
    't1Beam',     # stop time of interval for beam current evaluation
    't0Offset',   # start time of interval for beam current offset evaluation
    't1Offset'    # stop time of interval for beam current offset evaluation
]

df = pd.read_excel(dfs.master, usecols=cols, names=names, skiprows=1)
fdir = '../data/flux/'

# Progress bar
pb = IntProgress(min=0, max=df.count().values[0])
display(pb)

means, stds = [], []
offsetmeans, offsetstds = [], []


for i, row in df.iterrows():
    pb.value += 1
    data = pd.read_excel(fdir+row['tapeid']+'.xlsx', sheet_name=row['sname'], usecols=[1, 2], names=['time', 'ibeam'])

    cut = data[(row['t0Beam'] < data.time) & (data.time < row['t1Beam']) & (2e-10 < data.ibeam) & (data.ibeam != 0.0e0)]
    means.append(cut['ibeam'].mean()*1e9)
    stds.append(cut['ibeam'].std()*1e9)
    #print(row['t0Beam'], row['t1Beam'], cut['ibeam'].mean()*1e9, cut['ibeam'].std()*1e9)

    cut = data[(row['t0Offset'] < data.time) & (data.time < row['t1Offset']) & (data.ibeam != 0.0e0)]
    offsetmeans.append(cut['ibeam'].mean()*1e9)
    offsetstds.append(cut['ibeam'].std()*1e9)

    #print(row['t0Offset'], row['t1Offset'], cut['ibeam'].mean()*1e9, cut['ibeam'].std()*1e9)

data = {
    'mean': np.array(means),
    'std': np.array(stds),
    'omean': np.array(offsetmeans)
}

with pd.ExcelWriter(dfs.master, engine='openpyxl', if_sheet_exists='replace', mode='a') as writer:
    pd.DataFrame(data).to_excel(writer, index=False, sheet_name='output-ibeam', header=False)

os.system('say -v Yuna 안녕하세요. 코드가 완료되었습니다')

IntProgress(value=0, max=716)

0

## Step 2: extract Ic and n data for each "beam on event"

Here the values of Ic and n are extracted from the 3 Ic measurement files corresponding to measurements before, during and after irradiation. Some of the "beam on events" also have a temperature equivalent curve, which gives the temeprature required to match the IV curve measured during irradiation when the beam of off.

In [ ]:
ffunction = 'linear'
vMax = 20e-6

cols = [0, 15, 16, 17, 18]
names = [
    'tapeid',  # tape sample name
    'f0',      # filename of iv pristine
    'foff',    # filename of iv during beam exposure
    'fon',     # filename of iv before beam exposure
    'feq',     # filename of iv at temperature where iv matches iv-on
]

df = pd.read_excel(dfs.master, usecols=cols, names=names, skiprows=1)
icTs = pd.read_excel(dfs.master, sheet_name='IcT', usecols=[0, 1, 2, 3, 4, 5, 6, 7], skiprows=1)

# Progress bar
pb = IntProgress(min=0, max=df.count().values[0])
display(pb)

ic0, n0, icon, non, icoff, noff, iceq, neq = [], [], [], [], [], [], [], []
ic0err, n0err, icofferr, nofferr, iconerr, nonerr, iceqerr, neqerr = [], [], [], [], [], [], [], []
tHTS0, tTAR0, tHTSon, tTARon, tHTSoff, tTARoff, tHTSeq, tTAReq = [], [], [], [], [], [], [], []
tHTS0err, tTAR0err, tHTSonerr, tTARonerr, tHTSofferr, tTARofferr, tHTSeqerr, tTAReqerr = [], [], [], [], [], [], [], []
icCorrected, tCorrected = [], []

for i, row in df.iterrows():
    
    print('\nROW {}, {}\n'.format(i+3, row['tapeid']))
    
    for file in ['f0', 'fon', 'foff', 'feq']:
        
        
        ic, n, tHTS, tTAR = np.nan, np.nan, np.nan, np.nan
        icerr, nerr, tHTSerr, tTARerr = np.nan, np.nan, np.nan, np.nan
        icNoDegradation, tNoDegradation = np.nan, np.nan
        
        try:
            if 'interpolate' not in row[file]: # there is an IV file, we have a direct measurement but the ic won't
                fpath = '../data/Ic/{}/{}'.format(row.tapeid, row[file])
                ic, n, current, voltage, chisq, pcov = hts.fitIcMeasurement(fpath, fformat='mit', function=ffunction, vMax=vMax, vb=False)
                icerr, nerr = np.sqrt(pcov[0][0]), np.sqrt(pcov[1][1])
                data = hts.readIV(fpath)
                tHTS, tTAR = np.nanmean(data.sampleT[-10:]), np.nanmean(data.targetT[-10:])
                tHTSerr, tTARerr = np.nanstd(data.sampleT[-10:]), np.nanstd(data.targetT[-10:])
            
            if file == 'feq':
                if row['tapeid'] != 'f20':
                    _, icOnNoDegradation, tNoDegradation = bsp.getCorrectedSuppression(icon=icon[-1], icoff=icoff[-1], ic0=ic0[-1])
                else:
                    poptIcTc = [0.35584097, 0.77284068, 0.40751745, 0]
                    Tcp=88.43605636 
                    Tcd=76.45542551
                    _, icOnNoDegradation, tNoDegradation = bsp.getCorrectedSuppression(icon=icon[-1], icoff=icoff[-1], ic0=ic0[-1], Tcp=Tcp, Tcd=Tcd, poptIcTc=poptIcTc)
                    
        except ValueError as ve:
            print(ve, i+3, row.tapeid, row[file])
            tHTS = np.nan
        
        except TypeError as te:
            print(te)
        
        print('{}, {}'.format(file, ic))
        
        if file == 'f0':
            ic0.append(ic)
            ic0err.append(icerr)
            n0.append(n)
            n0err.append(nerr)
            tHTS0.append(tHTS)
            tHTS0err.append(tHTSerr)
            tTAR0.append(tTAR)
            tTAR0err.append(tTARerr)

        elif file == 'foff':
            icoff.append(ic)
            icofferr.append(icerr)
            noff.append(n)
            nofferr.append(nerr)
            tHTSoff.append(tHTS)
            tHTSofferr.append(tHTSerr)
            tTARoff.append(tTAR)
            tTARofferr.append(tTARerr)

        elif file == 'fon':
            icon.append(ic)
            iconerr.append(icerr)
            non.append(n)
            nonerr.append(nerr)
            tHTSon.append(tHTS)
            tHTSonerr.append(tHTSerr)
            tTARon.append(tTAR)
            tTARonerr.append(tTARerr)

        elif file == 'feq':
            iceq.append(ic)
            iceqerr.append(icerr)
            neq.append(n)
            neqerr.append(nerr)
            tHTSeq.append(tHTS)
            tHTSeqerr.append(tHTSerr)
            tTAReq.append(tTAR)
            tTAReqerr.append(tTARerr)
            
            icCorrected.append(float(icOnNoDegradation))
            tCorrected.append(float(tNoDegradation))

        pb.value += 1
        
data = {
    'ic0': ic0, 
    'ic0err': ic0err,
    'n0': n0, 
    'n0err': n0err,
    
    'icoff': icoff, 
    'icofferr': icofferr,
    'noff': noff, 
    'nofferr': nofferr, 
    
    'icon': icon, 
    'iconerr': iconerr, 
    'non': non, 
    'nonerrr': nonerr, 
    
    'iceq': iceq, 
    'iceqerr': iceqerr, 
    'neq': neq, 
    'neqerr': neqerr,
    
    'tHTS0': tHTS0, 
    'tHTS0err': tHTS0err,
    'tTAR0': tTAR0, 
    'tTAR0err': tTAR0err, 
    
    'tHTSoff': tHTSoff, 
    'tHTSofferr': tHTSofferr,
    'tTARoff': tTARoff, 
    'tTARofferr': tTARofferr,
    
    'tHTSon': tHTSon, 
    'tHTSonerr': tHTSonerr,
    'tTARon': tTARon, 
    'tTARonerr': tTARonerr,  
    
    'tHTSeq': tHTSeq, 
    'tHTSeqerr': tHTSeqerr, 
    'tTAReq': tTAReq, 
    'tTAReqerr': tTAReqerr,
    
    'icOn0': icCorrected,
    'tREBCO': tCorrected,
}

with pd.ExcelWriter(dfs.master, engine='openpyxl',if_sheet_exists='replace', mode='a') as writer:
    pd.DataFrame(data).to_excel(writer, sheet_name='output-icnt', index=False, header=False)

os.system('say -v Yuna 안녕하세요. 코드가 완료되었습니다')

## Step 4: Suppression vs Degradation data

In [ ]:
path = '../data/Ic/IcTc/'

data = pd.read_excel(dfs.master, sheet_name='data', usecols=[0, 1, 12, 13, 16, 27, 57, 58], names=['tapeid', 'anneal', 'energy', 'power', 'foff', 'icoff', 'suppression', 'suppressionErr'], skiprows=1)
df = pd.read_excel(dfs.master, sheet_name='TcIc', usecols=[0, 1, 2, 3], names=['tapeid', 'anneal', 'tcfname', 'icfname'], skiprows=1)
p = 0.02

means, stds, tcs, ics, points = [], [], [], [], []

for i, row in df.iterrows():
    
    ic, _, _, _, _, _ = hts.fitIcMeasurement(path+row.icfname, function='linear', vMax=20e-6)
    cut = (row.anneal == data.anneal)
    cut &= (row.tapeid == data.tapeid)
    cut &= ((1.-p)*ic <= data.icoff) & (data.icoff <= (1.+p)*ic)
    cut &= (35 <= data.power) & (data.power <= 37)
    
    #cut &= data.energy != 800
    tcs.append(hts.fitTcMeasurement('../data/Tc/{}/'.format(row.tapeid)+row.tcfname, wsz=30, vb=False))
    ics.append(ic)
    
    if len(data[cut].index) > 0:
        means.append(data[cut].suppression.mean())
        stds.append(data[cut].suppression.std())
        points.append(data[cut].index.values+3)
    else:
        means.append(np.nan)
        stds.append(np.nan)
        points.append(np.nan)

output = { 'tcs': tcs, 'ics': ics, 'means': means, 'stds': stds, 'pts': points }

with pd.ExcelWriter(dfs.master, engine='openpyxl',if_sheet_exists='replace', mode='a') as writer:
    pd.DataFrame(output).to_excel(writer, sheet_name='output-TcIc', index=False, header=False)
    